In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import tensorflow
import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.layers as layers
from keras.models import Model
from keras.preprocessing import image

from tensorflow.keras.regularizers import l2

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import cv2
from google.colab.patches import cv2_imshow


from keras import backend as K
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from sklearn.utils import shuffle

import numpy as np
import os
from PIL import Image
import scipy.io as sio

In [ ]:
from keras.initializers import glorot_uniform


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """ 
    An identity block.

    Parameters
    ----------
    input_tensor:

    kernel_size: int
        The kernel size of middle conv layer at main path.

    filters: list[int]
        The filters of 3 conv layer at main path.

    stage: int
        Current stage label, used for generating layer names.

    block: : str
        'a','b'..., current block label, used for generating layer names.

    Returns
    -------
    Output tensor for the block.
    """
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # for batch normalization layer, we assume
    # the input data is in channel last format
    bn_axis = 3

    filters1, filters2, filters3 = filters
  
    # main path, note that setting the kernel_initializer seed here is only used
    # for reproducibility, we techniqually don't need it
    x = layers.Conv2D(filters1, kernel_size=(1, 1), strides=(1, 1),
                      kernel_initializer=glorot_uniform(seed=0),
                      padding='valid', name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, strides=(1, 1),
                      kernel_initializer=glorot_uniform(seed=0),
                      padding='same', name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, kernel_size=(1, 1), strides=(1, 1),
                      kernel_initializer=glorot_uniform(seed=0),
                      padding='valid', name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    # this line is the core component of resnet, the skip connection, i.e.
    # having a shortcut to the main path before the activation, when addition
    # is performed on convolutional layers, the element-wise addition is performed
    # on their feature maps, i.e. channel by channel
    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x

In [ ]:
def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """ 
    A block that has a conv layer at shortcut.

    Parameters
    ----------
    input_tensor:
    
    kernel_size: int
        The kernel size of middle conv layer at main path.

    filters: list[int]
        The filters of 3 conv layer at main path.

    stage: int
        Current stage label, used for generating layer names.

    block: : str
        'a','b'..., current block label, used for generating layer names.
        
    strides : tuple, default (2, 2)
        Strides for the first conv layer in the block.

    Returns
    -------
    Output tensor for the block.
    """
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # for batch normalization layer, we assume
    # the input data is in channel last format,
    # which is the case if we are using the default
    # keras' backend tensorflow
    bn_axis = 3

    filters1, filters2, filters3 = filters
  
    # main path, note that setting the kernel_initializer set here is only used
    # for reproducibility, we techniqually don't need it
    x = layers.Conv2D(filters1, kernel_size=(1, 1), strides=strides,
                      kernel_initializer=glorot_uniform(seed=0),
                      padding='valid', name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters2, kernel_size, strides=(1, 1),
                      kernel_initializer=glorot_uniform(seed=0),
                      padding='same', name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters3, kernel_size=(1, 1), strides=(1, 1),
                      kernel_initializer=glorot_uniform(seed=0),
                      padding='valid', name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)
    
    # we resize the input so its dimension will match the output dimension
    # of the main path
    shortcut = layers.Conv2D(filters3, kernel_size=(1, 1), strides=strides,
                             kernel_initializer=glorot_uniform(seed=0),
                             padding='valid', name=conv_name_base + '1')(input_tensor)
    shortcut = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut) 

    # this line is the core component of resnet, the skip connection, i.e.
    # having a shortcut to the main path before the activation
    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

In [ ]:
trainpath="/content/gdrive/MyDrive/adv_orig_Dataset/mat_adv_orig_Dataset/mat_orig_cw_adv_Dataset/Training"
testpath="/content/gdrive/MyDrive/adv_orig_Dataset/mat_adv_orig_Dataset/mat_orig_cw_adv_Dataset/Testing"

In [ ]:
class_type= os.listdir(trainpath)

In [ ]:
class_type

['orig', 'adv']

In [ ]:
def load_dataset(path,class_type):
  labels = []
  mats = []
  for item in class_type:
    filenames = os.listdir(path + '/' + item)
    for f in filenames:
      mat_contents = sio.loadmat(path + '/' + item +'/' + f)
      if 'orig' in mat_contents:
        img=np.array((mat_contents['orig']*255).astype('uint8'))
        labels.append(0)
      else:
        img=np.array((mat_contents['adv']*255).astype('uint8'))
        labels.append(1)
      mats.append(np.array(img))  

  mats = np.array(mats)
  labels = np.array(labels) 
  labels = labels.reshape(-1, 1) 
  mats,labels = shuffle(mats, labels, random_state = 23)
  return mats,labels     

'''
  for filename in os.listdir(filepath):
    mat_contents = sio.loadmat(filepath+'/'+filename)
    if 'orig' in mat_contents:
      img=np.array((mat_contents['orig']*255).astype('uint8'))
      labels.append(0)
    else:
      img=np.array((mat_contents['adv']*255).astype('uint8'))
      labels.append(1)
    images.append(np.array(img/255))
  images = np.array(images)
  labels=np.array(labels)
  images,labels = shuffle(images,labels , random_state=4)
  '''

"\n  for filename in os.listdir(filepath):\n    mat_contents = sio.loadmat(filepath+'/'+filename)\n    if 'orig' in mat_contents:\n      img=np.array((mat_contents['orig']*255).astype('uint8'))\n      labels.append(0)\n    else:\n      img=np.array((mat_contents['adv']*255).astype('uint8'))\n      labels.append(1)\n    images.append(np.array(img/255))\n  images = np.array(images)\n  labels=np.array(labels)\n  images,labels = shuffle(images,labels , random_state=4)\n  "

In [ ]:
train_x,train_y=load_dataset(trainpath,class_type)

In [ ]:
test_x,test_y=load_dataset(testpath,class_type)

In [ ]:
train_y

array([[1],
       [1],
       [0],
       ...,
       [1],
       [1],
       [0]])

In [ ]:
n_classes = 2
img_rows, img_cols, img_channel = 224, 224, 3

# mnist is grey-scaled image, thus the last dimension, channel size will be 1
train_X = train_x.reshape(train_x.shape[0], img_rows, img_cols, img_channel)
test_X  = test_x.reshape(test_x.shape[0], img_rows, img_cols, img_channel)
input_shape = img_rows, img_cols, img_channel

train_X = train_X.astype('float32')
test_X  = test_X.astype('float32')

# images takes values between 0 - 255, we can normalize it
# by dividing every number by 255
train_X /= 255
test_X /= 255
print('train shape:', train_X.shape)

# one-hot encode the class (target) vectors
train_Y = np_utils.to_categorical(train_y, n_classes)
test_Y  = np_utils.to_categorical(test_y , n_classes)
#print('Y_train shape:', Y_test1_adv_orig.shape)

#Y_train
#Y_train1_adv_orig[0].shape
train_Y[0].shape

train shape: (1379, 224, 224, 3)


(2,)

In [ ]:
def ResNet(input_shape, n_classes):
    """
    Definition of ResNet
    
    References
    ----------
    https://github.com/keras-team/keras-applications/blob/master/keras_applications/resnet50.py
    """
    img_input = layers.Input(shape=input_shape)
    
    bn_axis = 3
    
    x = layers.ZeroPadding2D(padding=(3, 3), name='conv1_pad')(img_input)
    x = layers.Conv2D(64, (7, 7),
                      strides=(2, 2),
                      padding='valid',
                      kernel_initializer='he_normal',
                      name='conv1')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = layers.Activation('relu')(x)
    x = layers.ZeroPadding2D(padding=(1, 1), name='pool1_pad')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')
    
    # the commented out blocks are what's needed to build out the
    # full ResNet50 (a ResNet with 50 layers), we won't be needing
    # the complexity here
    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    # x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    # x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    # x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    # x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    # x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    # x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')
    
    x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    #img_output = layers.Dense(n_classes, activation='softmax', name='fc' + str(n_classes))(x)
    img_output = layers.Dense(2, kernel_regularizer=tensorflow.keras.regularizers.l2(0.01),activation
             ='softmax' , name = 'fc' + str(n_classes))(x); 
    model = Model(inputs=img_input, outputs=img_output, name='resnet')
    return model


model = ResNet(input_shape, 2)
model.compile(optimizer = 'adam', loss = 'squared_hinge', metrics = ['accuracy'])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#model.summary()

In [ ]:
history = model.fit(train_X, train_Y, batch_size=32, epochs=100, validation_data=(test_X,test_Y))

Epoch 1/100
44/44 [==============================] - 24s 552ms/step - loss: 1.5011 - accuracy: 0.5054 - val_loss: 1.5284 - val_accuracy: 0.4966
Epoch 2/100
44/44 [==============================] - 24s 551ms/step - loss: 1.4256 - accuracy: 0.4931 - val_loss: 1.5136 - val_accuracy: 0.5017
Epoch 3/100
44/44 [==============================] - 24s 550ms/step - loss: 1.3610 - accuracy: 0.5054 - val_loss: 1.3626 - val_accuracy: 0.4730
Epoch 4/100
44/44 [==============================] - 24s 551ms/step - loss: 1.3686 - accuracy: 0.5033 - val_loss: 1.3633 - val_accuracy: 0.5017
Epoch 5/100
44/44 [==============================] - 24s 550ms/step - loss: 1.4002 - accuracy: 0.5054 - val_loss: 1.3060 - val_accuracy: 0.4916
Epoch 6/100
44/44 [==============================] - 24s 550ms/step - loss: 1.3379 - accuracy: 0.4917 - val_loss: 1.4403 - val_accuracy: 0.5051
Epoch 7/100
44/44 [==============================] - 24s 550ms/step - loss: 1.2985 - accuracy: 0.5040 - val_loss: 1.2998 - val_accuracy:

In [ ]:
loss, accuracy = model.evaluate(test_x, test_y)
print('Loss = ' + str(loss))
print('Test Accuracy = ' + str(accuracy))